##Tarea 3

El objetivo de este taller es resolver el problema de clasificación de potabilidad de agua
usando los datos adjuntos. Divida apropiadamente los datos en conjuntos de entrenamiento y
prueba. A su vez, parte de los datos de entrenamiento se deben utilizar en los procedimientos
de selección de modelo.


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
# 1. Cargar los datos
datos = pd.read_csv('water_potability.csv').dropna()
print("Forma por defecto del dataset '.csv': \n", list(datos.keys()))

# 2. Preprocesar los datos si es necesario
X = datos.drop('Potability', axis=1)
y = datos['Potability']

# Lidiar con los valores nulos, normalizar, etc.
scaler = StandardScaler()
X_n= scaler.fit_transform(X)

# 3. Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_n, y, test_size=0.2, random_state=42)

Forma por defecto del dataset '.csv': 
 ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity', 'Potability']


#1.
Resuelva el problema utilizando Random Forest. Seleccione el modelo apropiado por
validación cruzada. para esto entrene modelos con los siguientes parámetros:


*   Número de árboles: Por lo menos 4 valores escogidos entre 100 y 500,
*   Mínimo número de datos por hoja: Por lo menos 3 valores entre 2 y 30.




In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
# Configurar la grilla de parámetros para Random Forest
params_RF = {
    'n_estimators'      :   [100, 200, 300, 500],  # Número de árboles
    'min_samples_leaf'  :            [2, 15, 30],  # Mínimo número de datos por hoja
}

# Crear y entrenar el clasificador Random Forest
RF = RandomForestClassifier(random_state=42)
cv_RF = GridSearchCV(estimator = RF, param_grid = params_RF) # Introducir los parámetros de validación cruzada
cv_RF.fit(X_train, y_train)

# Seleccionar el mejor modelo
mejor_RF = cv_RF.best_estimator_

# Evaluar el modelo seleccionado en el conjunto de prueba
y_pred_RF = mejor_RF.predict(X_test)

In [5]:
from sklearn.metrics import accuracy_score, classification_report

In [6]:
# Stats
mejores_parametros_RF = cv_RF.best_params_
accuracy_RF = accuracy_score(y_test, y_pred_RF)

# Printer
print("-@Random Forest-")
print("\n Mejores parámetros: ",
      "\n - Número de árboles (estimadores):   ", list(mejores_parametros_RF.values())[0],
      "\n - Número de hojas (datos)        : ",   list(mejores_parametros_RF.values())[1])
print("\n Accuracy en conjunto de prueba:   ",    accuracy_RF)
print("\n Reporte: \n",      classification_report(y_test, y_pred_RF))

-@Random Forest-

 Mejores parámetros:  
 - Número de árboles (estimadores):    2 
 - Número de hojas (datos)        :  500

 Accuracy en conjunto de prueba:    0.6699751861042184

 Reporte: 
               precision    recall  f1-score   support

           0       0.66      0.86      0.75       231
           1       0.69      0.41      0.52       172

    accuracy                           0.67       403
   macro avg       0.68      0.64      0.63       403
weighted avg       0.67      0.67      0.65       403



#2.
Resuelva el problema utilizando Xgboost. Seleccione el modelo apropiado por validación cruzada. para esto entrene modelos con los siguientes parámetros:


*   Número de árboles: Por lo menos 8 valores escogidos entre 50 y 600,
*   Factor de encogimiento: Por lo menos 5 valores entre 0 y 1.




In [7]:
from xgboost import XGBClassifier

In [8]:
# Configurar la grilla de parámetros para XGBoost
params_XGB = {
    'n_estimators'    : [50, 100, 150, 200, 250, 300, 400, 600],  # Número de árboles
    'learning_rate'   :              [0.05, 0.1, 0.2, 0.3, 0.4],  # Factor de encogimiento
}

# Crear y entrenar el clasificador XGBoost
XGB = XGBClassifier(use_label_encoder = False, eval_metric = 'logloss')
cv_XGB = GridSearchCV(estimator = XGB, param_grid = params_XGB) # Introducir los parámetros de validación cruzada
cv_XGB.fit(X_train, y_train)

# Seleccionar el mejor modelo
mejor_XGB = cv_XGB.best_estimator_

# Evaluar el modelo seleccionado en el conjunto de prueba
y_pred_XGB = mejor_XGB.predict(X_test)

In [9]:
# Stats
mejores_parametros_XGB = cv_XGB.best_params_
accuracy_XGB = accuracy_score(y_test, y_pred_XGB)

# Printer
print("-@XGBoost-")
print("\n Mejores parámetros: ",
      "\n - Número de árboles (estimadores)   :",           list(mejores_parametros_XGB.values())[0],
      "\n - Tasa de aprendizaje (encogimiento):  ",         list(mejores_parametros_XGB.values())[1])
print("\n Accuracy en conjunto de prueba:    ",    accuracy_XGB)
print("\n Reporte: \n",      classification_report(y_test, y_pred_XGB))

-@XGBoost-

 Mejores parámetros:  
 - Número de árboles (estimadores)   : 0.05 
 - Tasa de aprendizaje (encogimiento):   50

 Accuracy en conjunto de prueba:     0.630272952853598

 Reporte: 
               precision    recall  f1-score   support

           0       0.63      0.85      0.72       231
           1       0.62      0.34      0.44       172

    accuracy                           0.63       403
   macro avg       0.63      0.59      0.58       403
weighted avg       0.63      0.63      0.60       403



#3.
Escriba su propia implementación de Adaboost, usando como clasificador base decision
stumps (es decir árboles de decisión con sólo un nivel). Escriba su propia implementación de la rutina de entrenamiento de los clasificadores base. Entrene un modelo con
600 clasificadores base y grafique curvas de errores de entrenamiento y validación que
le permitan seleccionar un modelo con un número apropiado de clasificadores base.


In [ ]:
def MinimizarImpureza_feature(datos, feature):
    

#4.
Evalúe sus tres modelos resultantes en los datos de prueba con métricas apropiadas y
compare su desempeño.

# .
**Entregable:** Notebook de Jupyter en que se describen sus procedimientos, resultados,
análisis de resultados y conclusiones (y por supuesto, el código utilizado). Este informe
debe estar bien estructurado, con buena redacción y ortografía.
